#Human Detection

In [0]:
import numpy as np
from math import *
from PIL import Image
from copy import deepcopy
from scipy.misc import imsave
from matplotlib import pyplot as plt
import cv2
import glob

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def get_vector_for_cell(cell_M,cell_Theta,centres):
  #This function takes the cell gradient magnitude and gradient angle as input and generates 9 bins
  #corresponding to that cell and returns it
  
  res = [0 for i in range(9)]
  
  for u in range(8):
    for v in range(8):
      
      if cell_Theta[u,v]>160:
        res[-1] += ((20-(cell_Theta[u,v]-160))*cell_M[u,v])/20.0
        res[0] += ((cell_Theta[u,v]-160)*cell_M[u,v])/20.0
  
      elif cell_Theta[u,v]<0:
        res[0] += ((20-(0-cell_Theta[u,v]))*cell_M[u,v])/20.0
        res[-1] += ((0-cell_Theta[u,v])*cell_M[u,v])/20.0
    
      else:
        dist = []
        for x in range(len(centres)):
          dist.append((x,abs(centres[x]-cell_Theta[u,v])))

        dist.sort(key = lambda arr: arr[1]) 

        l = dist[:2]


        res[l[0][0]] += float(cell_M[u,v]*l[1][1])/20.0
        res[l[1][0]] += float(cell_M[u,v]*l[0][1])/20.0
    
  return res


def L2_norm(arr):
  #This function is used for L2 block normalization
  res = 0
  for ele in arr:
    res += (ele)**2
  return res**(0.5)


def flatten(arr):
  #This function is used to flatten 3d array to 1d array
  #This function is called to concatenate descriptors of 4 adjacent cells into one 
  #and also concatenate descriptors of all blocks into HOG descriptor
  res = []
  for u in range(arr.shape[0]):
    for v in range(arr.shape[1]):
      res += arr[u,v].tolist()
      
  return np.array(res)



def relu(arr):
  #This is the relu activation function used for hidden layer neurons
  res = deepcopy(arr)
  res[res<0]=0
  return res

def sigmoid(arr):
  #This is the sigmoid activation function for outer layer neuron
  return 1.0/(1+np.exp(-arr))

def MSE(arr1,arr2):
  #This function is used to find squared error
  return (0.5*(arr1-arr2)**2).sum()

def MSE_grad(arr1,arr2):
  #This function is the gradient of squared error function and is used during backpropagation
  return (arr2-arr1)

def sigmoid_grad(arr):
  #This function is the gradient of sigmoid activation function and is used during backpropagation
  return np.multiply(sigmoid(arr),1-sigmoid(arr))

def relu_grad(a):
  #This function is the gradient of relu activation function and is used during backpropagation
  arr = deepcopy(a)
  arr[arr<0]=0
  arr[arr>0]=1
  return arr

In [0]:
def rgb_to_gray_img(rgb_image):
  #This function converts an RGB image to grayscale one
  grayscale_image = np.zeros((rgb_image.shape[0],rgb_image.shape[1]))
  for i in range(rgb_image.shape[0]):
    for j in range(rgb_image.shape[1]):
        r,g,b= rgb_image[i,j]
        I = round(0.299*r + 0.587*g + 0.114*b)
        grayscale_image[i,j] = I
  return grayscale_image

In [0]:
def get_horizontal_and_vertical_gradients(grayscale_image):
  #This function returns the horizontal and vertical gradients of the grayscale_image
  Gx = np.zeros((grayscale_image.shape[0],grayscale_image.shape[1]))
  Gy = np.zeros((grayscale_image.shape[0],grayscale_image.shape[1]))
  Gx_mask = np.matrix([[-1,0,1],[-1,0,1],[-1,0,1]])
  Gy_mask = np.matrix([[1,1,1],[0,0,0],[-1,-1,-1]])


  for i in range(1,grayscale_image.shape[0]-1):            
      for j in range(1,grayscale_image.shape[1]-1):
        Gx[i,j] = (np.multiply(grayscale_image[i-1:i+2,j-1:j+2],Gx_mask).sum())
        Gy[i,j] = (np.multiply(grayscale_image[i-1:i+2,j-1:j+2],Gy_mask).sum())


  Gx = np.array(Gx)
  Gy = np.array(Gy)
  return [Gx,Gy]

In [0]:
def get_magnitude_and_gradient_angle(Gx,Gy):
  #This function returns the Normalized Gradient Magnitude and Gradient angles(in degrees) corresponding to 
  #the horizontal and vertical gradients of the grayscale_image
  M = np.zeros((Gx.shape[0],Gx.shape[1]))
  Theta = np.zeros((Gx.shape[0],Gx.shape[1]))

  for i in range(1,Gx.shape[0]-1):            
      for j in range(1,Gx.shape[1]-1):
        M[i,j] = sqrt((Gx[i,j])**2 + (Gy[i,j])**2)
        if Gx[i,j]==0 and Gy[i,j]==0:               #If both Gx and Gy are 0 then gradient angle is 0 degrees
          t = 0
        elif Gx[i,j] == 0:                            # This is to solve divide by 0 error
           t = 90                                   # If only Gx is 0, gradient angle is 90 degrees
        else:
          t = degrees(np.arctan(Gy[i,j]*1.0/Gx[i,j]))
          if t<0:                                   # All negative angles are shifted
            t += 180                                # to the range of 90 and 180 degrees this range is [0,180)
        if t>=170:                        #This is done to get the gradient angle range from [0,180)                            
          t -= 180                        #to [-10,170). This is achieved by subtracting 180 from all angles 
        Theta[i,j] = t                    # greater than or equal to 170

  M = np.array(M)
  Theta = np.array(Theta)
  return [M,Theta]

In [0]:
def get_cells(M,Theta):
  #This function generates a 2d array of cells, with each element being a bin of length 9
  cells = [[[0 for k in range(9)] for j in range(M.shape[1]/8)] for i in range(M.shape[0]/8)]
  for i in range(0,M.shape[0],8):
    for j in range(0,M.shape[1],8):
      cell_M = M[i:i+8,j:j+8]
      cell_Theta = Theta[i:i+8,j:j+8]

      cells[i/8][j/8] = get_vector_for_cell(cell_M,cell_Theta,[(c*20) for c in range(9)])
  cells = np.array(cells)
  return cells

In [0]:
def get_blocks(cells):
  #This function generates a 2d array of block, with each element being a descriptor of length 36
  blocks = [[[0 for k in range(36)] for j in range(cells.shape[1]-1)] for i in range(cells.shape[0]-1)]
  for i in range(0,cells.shape[0]-1):
    for j in range(0,cells.shape[1]-1):

      temp = flatten(cells[i:i+2,j:j+2])
      if float(L2_norm(temp)) != 0:
        blocks[i][j] = (temp/float(L2_norm(temp))).tolist()

  blocks = np.array(blocks)
  return blocks

In [0]:
def get_final_descriptor_from_blocks(blocks):
  #This function generates the HOG descriptor of length 7524 by flattening/concatenating all the block descriptors
  return flatten(blocks)

In [0]:
def get_final_descriptor(images):
  #This function generates HOG descriptor for each image from the list of images
  # and creates a 2d array, where each row corresponds to an image and 7524 columns representing HOG descriptors
  final_descriptors1 = []
  
  for i in range(len(images)):
    rgb_image1 = images[i]
    grayscale_image1 = rgb_to_gray_img(rgb_image1)
    Gx1 , Gy1 = get_horizontal_and_vertical_gradients(grayscale_image1)
    M1 , Theta1 = get_magnitude_and_gradient_angle(Gx1,Gy1)
    if len(images)==10:
      imsave('Test_{}.bmp'.format(i), np.array(M1))
    cells1 = get_cells(M1,Theta1)
    blocks1 = get_blocks(cells1)
    final_descriptor1 = get_final_descriptor_from_blocks(blocks1)
    if len(images)==20 and i==5:
      np.savetxt('HogDescriptor_crop001278a.txt', final_descriptor1, delimiter='\n')
    elif len(images)==10 and i==2:
      np.savetxt('HogDescriptor_crop001045b.txt', final_descriptor1, delimiter='\n')     
    final_descriptors1.append(final_descriptor1)
  final_descriptors1 = np.array(final_descriptors1)
  return final_descriptors1

In [0]:
def get_weights(input_layer_neurons,hidden_layer_neurons):
  #Random initialization of weights and bias
  w = np.random.normal(0, 0.1, size=(input_layer_neurons, hidden_layer_neurons))
  v = np.random.normal(0, 0.1, size=(hidden_layer_neurons, 1))

  b1 = np.random.normal(0, 0.1, size=(1,hidden_layer_neurons))
  b2 = np.random.normal(0, 0.1, size=(1,1))
  return [w,v,b1,b2]

In [0]:
def training(X,y_actual,w,v,b1,b2,lr):
  #This function is for training the Multi Layer Perceptron Model on the given training data and random weights and bias
  #This function returns the epochs,Error,updated weights and bias when the Errors in two consecutive epochs are same after 
  #rounding them to 5 places after decimal
  epochs = 0
  E_prev = float('inf')
  while True:
    E = 0
    for i in range(X.shape[0]):
      x = X[i].reshape(1,-1) 
      z = np.dot(x,w) + b1
      z = relu(z)

      y = np.dot(z,v) + b2
      y = sigmoid(y)
      E += MSE(y_actual[i],y)

      grad = MSE_grad(y_actual[i],y)
      grad = np.multiply(grad,sigmoid_grad(y))
      vd = np.dot(grad.T,z).T
      b2d = np.dot(grad.T,np.ones((z.shape[0],1))).T

      grad = np.dot(grad,v.T)
      v -= lr*vd
      b2 -= lr*b2d 

      grad = np.multiply(grad,relu_grad(z))
      wd = np.dot(grad.T,x).T
      b1d = np.dot(grad.T,np.ones((x.shape[0],1))).T

      grad = np.dot(grad,w.T)
      w -= lr*wd
      b1 -= lr*b1d
    epochs += 1
    if round(E,5) == round(E_prev,5):
      return [epochs,E,w,v,b1,b2]
    else:
      E_prev = E
    

In [0]:
def testing(X,w,v,b1,b2):
  #This function is used to test the images and generate output after applying sigmoid activation function
  z = np.dot(X,w) + b1
  z = relu(z)
  y = np.dot(z,v) + b2
  y = sigmoid(y)
  return y

In [0]:
train_images = [cv2.imread(file) for file in glob.glob("/content/drive/My Drive/Human/Train_Positive/*.bmp")]
train_images += [cv2.imread(file) for file in glob.glob("/content/drive/My Drive/Human/Train_Negative/*.bmp")]
train_final_descriptors = get_final_descriptor(train_images)


test_images = [cv2.imread(file) for file in glob.glob("/content/drive/My Drive/Human/Test_Positive/*.bmp")]
test_images += [cv2.imread(file) for file in glob.glob("/content/drive/My Drive/Human/Test_Neg/*.bmp")]
test_final_descriptors = get_final_descriptor(test_images)


y_train = np.array([1 for i in range(10)]+[0 for i in range(10)]).reshape(-1,1)
y_test = np.array([1 for i in range(5)]+[0 for i in range(5)]).reshape(-1,1)

##Classification of test images for different learning rates :-

###When learning rate = 0.2,

In [0]:
for hidden_layer_neurons in [250,500,1000]:
  print 'For {} hidden layer neurons,'.format(hidden_layer_neurons)
  w,v,b1,b2 = get_weights(train_final_descriptors.shape[1],hidden_layer_neurons)
  epochs,E,w,v,b1,b2 = training(train_final_descriptors,y_train,w,v,b1,b2,0.18)
  y_pred = testing(test_final_descriptors,w,v,b1,b2)
  print 'Epochs = {}'.format(epochs)
  print 'Error = {}'.format(E)
  print 'Output value = {}'.format(y_pred.reshape(1,-1))
  y_pred[y_pred>=0.5]=1
  y_pred[y_pred<0.5]=0
  print 'Classification = {}'.format(y_pred.reshape(1,-1))
  print 'Accuracy = {}%'.format((y_pred[y_pred==y_test].shape[0])*100.0/y_pred.shape[0])
  print '\n'


For 250 hidden layer neurons,
Epochs = 13
Error = 2.05342006981e-05
Output value = [[0.93257423 0.99993987 0.57244634 0.97617415 0.05211053 0.20249758
  0.04281609 0.00148125 0.04123898 0.09247734]]
Classification = [[1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]]
Accuracy = 90.0%


For 500 hidden layer neurons,
Epochs = 14
Error = 7.87607452205e-06
Output value = [[9.94477900e-01 9.99993858e-01 9.90142280e-01 9.91103416e-01
  9.20436497e-02 1.15706431e-01 2.58671639e-02 3.65888089e-05
  2.15566512e-02 4.06752746e-02]]
Classification = [[1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]]
Accuracy = 90.0%


For 1000 hidden layer neurons,
Epochs = 12
Error = 1.82191539945e-06
Output value = [[9.99996453e-01 1.00000000e+00 9.99999991e-01 9.99999718e-01
  5.01915777e-02 1.53693468e-01 1.36580560e-01 6.31815219e-11
  2.49422285e-01 2.09296777e-04]]
Classification = [[1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]]
Accuracy = 90.0%




###When learning rate = 0.1,

In [0]:
for hidden_layer_neurons in [250,500,1000]:
  print 'For {} hidden layer neurons,'.format(hidden_layer_neurons)
  w,v,b1,b2 = get_weights(train_final_descriptors.shape[1],hidden_layer_neurons)
  epochs,E,w,v,b1,b2 = training(train_final_descriptors,y_train,w,v,b1,b2,0.1)
  y_pred = testing(test_final_descriptors,w,v,b1,b2)
  print 'Epochs = {}'.format(epochs)
  print 'Error = {}'.format(E)
  print 'Output value = {}'.format(y_pred.reshape(1,-1))
  y_pred[y_pred>=0.5]=1
  y_pred[y_pred<0.5]=0
  print 'Classification = {}'.format(y_pred.reshape(1,-1))
  print 'Actual Labels = {}'.format(y_test.reshape(1,-1))
  print 'Accuracy = {}%'.format((y_pred[y_pred==y_test].shape[0])*100.0/y_pred.shape[0])
  print '\n'


For 250 hidden layer neurons,
Epochs = 55
Error = 1.95579233576e-05
Output value = [[0.85104257 0.9976786  0.96697477 0.93677706 0.11415916 0.28361686
  0.11297893 0.00496342 0.38637148 0.07748381]]
Classification = [[1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]]
Actual Labels = [[1 1 1 1 1 0 0 0 0 0]]
Accuracy = 90.0%


For 500 hidden layer neurons,
Epochs = 12
Error = 2.01414542303e-08
Output value = [[8.43173744e-01 9.99989284e-01 9.94938749e-01 9.99798024e-01
  6.39347672e-01 3.94389500e-02 4.18083709e-02 4.66437368e-05
  4.41191969e-02 1.23899519e-02]]
Classification = [[1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]]
Actual Labels = [[1 1 1 1 1 0 0 0 0 0]]
Accuracy = 100.0%


For 1000 hidden layer neurons,
Epochs = 22
Error = 4.74492349554e-06
Output value = [[9.99989451e-01 9.99999984e-01 9.87248432e-01 9.99996573e-01
  9.96177490e-01 6.26905812e-02 1.13085164e-01 2.76787401e-06
  7.17418612e-01 7.96958751e-02]]
Classification = [[1. 1. 1. 1. 1. 0. 0. 0. 1. 0.]]
Actual Labels = [[1 1 1 1 1 0 0 0 0 0]]
Accura

###When learning rate = 0.15,

In [0]:
for hidden_layer_neurons in [250,500,1000]:
  print 'For {} hidden layer neurons,'.format(hidden_layer_neurons)
  w,v,b1,b2 = get_weights(train_final_descriptors.shape[1],hidden_layer_neurons)
  epochs,E,w,v,b1,b2 = training(train_final_descriptors,y_train,w,v,b1,b2,0.15)
  y_pred = testing(test_final_descriptors,w,v,b1,b2)
  print 'Epochs = {}'.format(epochs)
  print 'Error = {}'.format(E)
  print 'Output value = {}'.format(y_pred.reshape(1,-1))
  y_pred[y_pred>=0.5]=1
  y_pred[y_pred<0.5]=0
  print 'Classification = {}'.format(y_pred.reshape(1,-1))
  print 'Accuracy = {}%'.format((y_pred[y_pred==y_test].shape[0])*100.0/y_pred.shape[0])
  print '\n'


For 250 hidden layer neurons,
Epochs = 11
Error = 0.000418618359343
Output value = [[0.97437874 0.99993694 0.97535996 0.99738982 0.53089917 0.0665184
  0.09006134 0.00396099 0.29947957 0.17736695]]
Classification = [[1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]]
Accuracy = 100.0%


For 500 hidden layer neurons,
Epochs = 11
Error = 9.12175710951e-05
Output value = [[8.80168510e-01 9.99951945e-01 9.97560409e-01 9.98920623e-01
  8.98300273e-01 3.00897247e-02 5.59561419e-02 9.41292797e-04
  1.63365534e-02 2.05094362e-02]]
Classification = [[1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]]
Accuracy = 100.0%


For 1000 hidden layer neurons,
Epochs = 12
Error = 2.62123226867e-14
Output value = [[9.99997716e-01 1.00000000e+00 9.99999994e-01 9.99999999e-01
  9.99172011e-01 7.66887747e-04 3.82090427e-02 3.67318407e-10
  3.83665307e-02 7.78593579e-04]]
Classification = [[1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]]
Accuracy = 100.0%


